In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
from simulator import FdaSimulator

In [ ]:
fda_simulator = FdaSimulator()

In [ ]:
cnt = 30
alpha_p =  1 * cnt
beta_p = 3 * cnt
# "linear_unimodal" (200, 1), "linear_bimodal" (200, 1), "non_linear_unimodal" (200, 1), "non_linear_bimodal" (200, 1), "bct"

X, phi_X, epsilon, beta_data, col_indexes_bct, target = fda_simulator.simulate(
    type_covariate="brownian_with_a_trend",
    type_transformation="bct",
    sd_x=0.01,
    sample_size=200,
    eta=0.05,
    # Series representation
    n_basis_simulated_data=5,
    # Beta parameters
    alpha_param=alpha_p,
    beta_param=beta_p,
    # Brownian parameters
    intercept_brownian=0,
    slope_brownian=1,
    positions=[0.15, 0.35, 0.55, 0.85]
)

In [ ]:
X_data_series_grid = fda_simulator.to_fdata_grid(X)
zzz = X_data_series_grid.plot()

In [ ]:
if not beta_data is None:
    beta_data_grid = fda_simulator.to_fdata_grid(beta_data)
    plt.plot(
        fda_simulator.abscissa_points,
        np.reshape(beta_data, newshape=(1, -1))[0], '-o'
    )

In [ ]:
epsilon2 = np.add(target, -phi_X)
np.var(epsilon)/(np.var(epsilon) + np.var(phi_X))

In [ ]:
np.max(np.abs(epsilon - epsilon2))